In [39]:
from abc import ABC, abstractmethod

class Reality():


    def __init__(
            self, person_list=[], broker_list=[], index_list=[], starting_time_period=0):
        self.person_list = person_list
        self.broker_list = broker_list
        self.index_list = index_list
        self.time_period = starting_time_period
        self.person_data = {}
        self.index_data = {}
        self.t = starting_time_period

    def execute_person_behaviour(self, person, behaviour_dict,
                                  income_change=0, acc_list=None, acc_index_dict=None):
        """Investing behaviour of person is always executed at the beginning of the period
        """

        person.change_monthly_income(income_change=income_change)
        person.receive_monthly_income()
        for acc in acc_list:
            person.add_account(booker_account=acc)
        for acc in acc_index_dict:
            for index in acc_index_dict[acc]:
                person.add_instrument_to_account(account=acc, index=index)
        person.execute_investing_behaviour(behaviour_dict=behaviour_dict)
        self.save_person_data(person)

    def calculate_accs_totals(self, person):
        """Account totals of the person are calculated at the end of the period
        """

        for account in person.broker_accs:
            account.calculate_inst_amount_wraper()
        self.save_person_data(person)

    def calculate_all_peeople_accs_totals(self):
        for person in self.person_list:
            self.calculate_accounts_amount(person=person)

    def execute_indexes_period(self):
        for index in self.index_list:
            self.execute_index_period(index)

    def execute_index_period(self, index):
        index.execute_time_period()
        self.save_index_data(index=index)

    def execute_people_behaviour(self, behaviour_dict):
        for person in self.person_list:
            self.execute_person_behaviour(person=person,
                                           behaviour_dict=behaviour_dict[person.name])

    def save_person_data(self, person):
        if not self.person_data[person]:
              self.person_data[person] = {}
        self.person_data[person][self.t] = {}
        self.person_data[person][self.t]['amount'] = person.amount
        self.person_data[person][self.t]['income'] = person.current_monthly_income
        self.person_data[person][self.t]['age'] = person.age
        for account in person.broker_accs:
            self.person_data[person][self.t]['accounts'][account.booker_name] = {}
            self.person_data[person][self.t]['accounts'][account.booker_name]['amount'] = account.amount
            self.person_data[person][self.t]['accounts'][account.booker_name]['instruments'] = account.index_dict
    
    def save_index_data(self, index):
        self.index_data[index.name][self.t] = index.current_price

    def execute_period(self):
        """changes to the accounts/indexes employed and adding money to accounts are decided at the beginning of the period
        calculation of account balances then happens at the end of the period"""

        self.t += 1
        self.execute_people_behaviour()
        self.execute_indexes_period()
        self.calculate_all_peeople_accs_totals()


class Person():

    def __init__(
            self, name, starting_age, starting_money, current_monthly_income, reality, broker_acounts=[]):
        self.name = name
        self.age = starting_age
        self.money = starting_money
        self.current_monthly_income = current_monthly_income 
        self.broker_accs = broker_acounts
        self.investing_behaviour = {}
        self.reality = reality

    def add_account(self, broker_name):
        if broker_name not in self.reality.broker_list:
            print("Broker does not exist in this reality")
            return
        account = Account(owner=self.name, broker_name=broker_name)
        self.broker_accs.append(account)

    def delete_account(self, account):
        self.broker_accs.remove(account)

    def add_index_to_account(self, account_name, index):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to the person')
            return
        acc_index =  self.broker_acc.index(acc) 
        self.broker_accs[acc_index].index_dict[index.name] = 0

    def add_money_to_account(self, account_name, money):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to this person')
            return
        acc_index =  self.broker_acc.index(acc)
        self.broker_accs[acc_index].account_amount += money

    def take_money_from_account(self, account_name, money):
        acc = self.get_account_by_name(account_name=account_name)
        if acc is None:
            print('Account does not belong to this person')
            return
        acc_index =  self.broker_acc.index(acc)
        self.broker_accs[acc_index].account_amount -= money
        self.money += money

    def add_money_to_index(self, account, index, money):
        account.transfer_money_to_instrument(index=index.name, money=money)
        
    def receive_monthly_income(self):
        self.money = self.money + self.current_monthly_income

    def change_monthly_income(self, income_change):
        self.current_monthly_income = (self.current_monthly_income  
                                       + income_change)
                
    def execute_investing_behaviour(self, behaviour_dict):
        for account in behaviour_dict:
            account_total = sum(behaviour_dict[account].values())
            self.add_money_to_account(account=account_total,
                                       money=account_total
                                       *self.current_monthly_income)
            for index in account:
                self.add_money_to_index(account=account, instrument=index, 
                                        money=(
                                            self.current_monthly_income
                                            *behaviour_dict[account][index]))
                
    def get_account_by_name(self, account_name):
        print(account_name)
        for account in  self.broker_accs:
            print(account.broker_name)
            if account.broker_name == account_name:
                return account

        
class Account():

    def __init__(self, owner, broker_name, starting_index_dict={}, starting_amount=0):
        self.broker_name = broker_name
        self.owner = owner
        self.index_dict = starting_index_dict
        self.account_amount  = starting_amount

    def add_instrument(self, index):
        self.index_dict[index.name] = 0
    
    def delete_instrument(self, index):
        if self.index_dict[index.name] != 0:
            print('can delete account, there are still money in it')
            return
        del self.index_dict[index.name]

    def calculate_instrument_amount(self, index_name):
        if index_name not in self.index_dict:
            print('Index is not active in current account')
            return
        curr_price = self.owner.reality.index_data[index_name][-1]
        prev_price = self.owner.reality.index_data[index_name][-2]
        multip = (curr_price - prev_price) / (curr_price)
        self.index_dict[index_name] = self.index_dict[index_name] * multip

    def calculate_inst_amount_wraper(self):
        for index_name in self.index_dict:
            self.calculate_instrument_amount(index_name=index_name)

    def transfer_money_to_instrument(self, instrument, money):
        if money > self.account_amount:
            print('Not enough money for the transaction')
            return
        self.account_amount = self.account_amount - money
        self.index_dict[instrument.name] += money


class  Instrument(ABC):

    def __init__(self, name, type_):
        self.name = name
        self.type = type_
        self.multip = 1

    @abstractmethod
    def calculate_multip(self):
        pass

    @abstractmethod
    def execute_time_period(self):
        pass


class Index(Instrument):

    def __init__(
            self, name, reality, type_='index', price_ts=[], current_price=None, previous_price=None):
        super().__init__(type_, name)
        self.current_price = current_price
        self.previous_price = previous_price
        self.price_ts = price_ts
        self.reality = reality

    def calculate_current_price(self, t, mma_length):
        self.previous_price = self.current_price 
        self.current_price = self.ts_calc(mma_length)
        self.price_ts[t] = self.current_price
        self.reality

        pass

    def ts_calc(self, mma_length):
        if len(self.price_ts) < mma_length:
            return('We need at least ' 
                  + str(mma_length) 
                  + " previous observations")
        current_price = ((self.price_ts[len(self.price_ts) - (mma_length): ] 
                          / (mma_length)))
        return current_price
            
    def calculate_multip(self):
        (self.current_price - self.previous_price) // self.previous_price

    def execute_time_period(self):
        self.calculate_current_price()
        self.calculate_multip()


class TransactionAccount(Instrument):

    def __init__(self, name, type_='trans_account'):
        super().__init__(type_, name)

    def execute_time_period(self):
        pass




        



 

In [9]:
import pandas as pd
sap500 = pd.read_csv('sap500.csv')
sap500 = list(sap500["Close"])
sap500.reverse()

In [40]:
reality_1 = Reality()
reality_1.broker_list.append('kb')
nvesting_behaviour = {'Terka': {'kb': {'SAP500': 0.7}}}
index_sap500 = Index(name='SAP500', reality=reality_1, price_ts=sap500)
terka = Person(name='Terka', starting_age=26, starting_money=35000,
               current_monthly_income=25000, reality=reality_1)
terka.add_account('kb')
print(terka.broker_accs)
terka.add_index_to_account(terka.broker_accs[0], index=index_sap500)
for t in range(500):
    reality_1.execute_period()

kb
Account does not belong to the person


AttributeError: 'Reality' object has no attribute 't'

In [5]:
a = Index(name='aa')

In [13]:
reality = Reality()
person = Person(name='ja', starting_age=25, starting_money=0, reality=reality, current_monthly_income=1250)

In [14]:
reality.broker_list.append('kb')

In [16]:
person.reality.broker_list

['kb']

In [10]:
person.reality.broker_list

In [4]:
ind_1 = Index(name='SAP', type_='etf')
acc = Account()